In [14]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob

from PhicoreModules import parse_genbank, median, mean, stdev, mode
from PhicoreModules import get_features_of_type, get_features_lengths, get_gc_content, get_coding_density, get_distribution_of_stops # Przemek's functions
from PhicoreModules import get_mean_cds_length_rec_window, get_rolling_gc, get_rolling_mean_cds # George's functions
from PhicoreModules import write_df_to_artemis

## Work in progress on STOP codons entropy per frame

In [15]:
from typing import Union
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def get_codon_num_in_frame(seq: Union[str, SeqRecord, Seq], codon: str) -> int:
    """
    Count the number of codons in a sequence in the first frame.
    :param: seq: nucleotide sequence
    :param: codon: codon to count
    :return: number of codons
    """

    codons_num = 0
    for i in range(0, len(seq), 3):
        if seq[i:i+3] == codon:
            codons_num += 1
    return codons_num


def get_distribution_of_stops_per_frame(seqiorec, window=210, step=30):
    """
    Get distribution of stops
    :param seqiorec:
    :param window:
    :param step:
    :return:
    """

    stops = ['TAA', 'TAG', 'TGA']

    stops_frame_distr = {
        'x': range(1, len(seqiorec.seq) + 1),

        'frame1-TAA': [np.NAN]*int(window/2),
        'frame1-TAG': [np.NAN]*int(window/2),
        'frame1-TGA': [np.NAN]*int(window/2),

        'frame2-TAA': [np.NAN]*int(window/2),
        'frame2-TAG': [np.NAN]*int(window/2),
        'frame2-TGA': [np.NAN]*int(window/2),

        'frame3-TAA': [np.NAN]*int(window/2),
        'frame3-TAG': [np.NAN]*int(window/2),
        'frame3-TGA': [np.NAN]*int(window/2)
    }
    
    i = 0
    while i + window/2 + 3 <= len(seqiorec.seq) - window/2:
        window_seq = seqiorec.seq[i : i + window]
        for frame in range(3):
            taa = get_codon_num_in_frame(seqiorec.seq[i + frame: i + window + frame], 'TAA')
            tag = get_codon_num_in_frame(seqiorec.seq[i + frame : i + window + frame],'TAG')
            tga = get_codon_num_in_frame(seqiorec.seq[i + frame : i + window + frame],'TGA')
            stops_frame_distr['frame{}-TAA'.format(frame + 1)].extend([taa]*(step))
            stops_frame_distr['frame{}-TAG'.format(frame + 1)].extend([tag]*(step))
            stops_frame_distr['frame{}-TGA'.format(frame + 1)].extend([tga]*(step))
        i += step
        
    i -= step
    left = len(seqiorec.seq) - len(stops_frame_distr['frame1-TAA'])
    if left > 0:
        for frame in range(3):
            stops_frame_distr['frame{}-TAA'.format(frame + 1)].extend([np.NAN]*left)
            stops_frame_distr['frame{}-TAG'.format(frame + 1)].extend([np.NAN]*left)
            stops_frame_distr['frame{}-TGA'.format(frame + 1)].extend([np.NAN]*left)

    return pd.DataFrame(stops_frame_distr)

In [16]:
from typing import List
def get_all_stats(infile : str, outdir : str, plot : List[str] = 0,  write : List[str] = []):
    """
    Prints all stats we have so far on a single file
    :param infile: path to input file
    :param outdir: path to output directory
    :param plot: which plots to generate
    :param write: which stats and plots to write to file
    :return:
    """

    stats = {
        'infile': infile,
        'id': '', 
        'length': 0, 
        'gc': 0, 
        'cds_num': 0, 
        'cds_avg_len': 0, 
        'cds_med_len': 0, 
        'cds_tRNA_density': 0, 

    }

    print("=== Working on file {} ===".format(infile))
    for record in parse_genbank(infile):
        stats['id'] = record.id
        stats['length'] = len(record.seq)
        stats['gc'] = get_gc_content(record.seq)
        cdss = get_features_of_type(record, 'CDS')
        cdss_lengths = get_features_lengths(record, 'CDS')
        stats['cds_num'] = len(cdss)
        stats['cds_avg_len'] = mean(cdss_lengths)
        stats['cds_med_len'] = median(cdss_lengths)
        stats['cds_tRNA_density'] = get_coding_density(record, ['CDS', 'tRNA'])

        # output files
        df_stops_file = os.path.join(outdir, '{}.{}_stops.txt'.format(os.path.basename(infile), record.id))
        df_stops_per_frame_file = os.path.join(outdir, '{}.{}_stops_per_frame.txt'.format(os.path.basename(infile), record.id))
        df_stops_plot_file = os.path.join(outdir, '{}.{}_stops.png'.format(os.path.basename(infile), record.id))
        df_stops_per_frame_plot_file = os.path.join(outdir, '{}.{}_stops_per_frame.png'.format(os.path.basename(infile), record.id))
        # data
        window = 300
        step = 30

        if plot or write:
            print(" = Getting overall distribution of stops")
            df = get_distribution_of_stops(record, window, step)
            if 'stop_distr' in plot:
                df.plot(
                    x="x", 
                    y=df.columns[1:], 
                    figsize=(40, 8), 
                    legend=True, 
                    title="{} - {} # window: {}, step: {}".format(os.path.basename(infile), record.id, window, step)
                )
            if 'stop_distr' in write:
                print("Writing distribution of stops to {}".format(df_stops_file))
                write_df_to_artemis(df, df_stops_file)
                print("Writing plot of the distribution to {}".format(df_stops_plot_file))
                plt.savefig(df_stops_plot_file)

            if 'stop_distr_per_frame' in plot:
                print(" = Getting distribution of stops per frame")
                df = get_distribution_of_stops_per_frame(record, window, step)
                df.plot(
                    x="x", 
                    y=df.columns[1:], 
                    figsize=(40, 8), 
                    legend=True, 
                    title="{} - {} # window: {}, step: {}".format(os.path.basename(infile), record.id, window, step)
                )
            if 'stop_distr_per_frame' in write:
                print("Writing distribution of stops to {}".format(df_stops_file))
                write_df_to_artemis(df, df_stops_per_frame_file)
                print("Writing plot of the distribution to {}".format(df_stops_plot_file))
                plt.savefig(df_stops_per_frame_plot_file)
    
    return stats



In [17]:
infiles = glob.glob('./genbank/Bc11*')
outdir = './tmp'
df = pd.DataFrame()
for infile in infiles:
    df = df.append(get_all_stats(infile, outdir), ignore_index=True)

df

# for record in parse_genbank('./genbank/OFRY01000050-TAG-TGA-TAA.gbk'):
#     df_all = get_rolling_mean_cds(record, window=1000, step=30)
# for record in parse_genbank('./genbank/OFRY01000050-TGA-TAA.gbk'):
#     df_tag = get_rolling_mean_cds(record, window=1000, step=30)
# for record in parse_genbank('./genbank/OFRY01000050-TAG-TAA.gbk'):
#     df_tga = get_rolling_mean_cds(record, window=1000, step=30)
# for record in parse_genbank('./genbank/OFRY01000050-TAG-TGA.gbk'):
#     df_taa = get_rolling_mean_cds(record, window=1000, step=30)


# df_all['No_TAG'] = df_tag['Mean_CDS']
# df_all['No_TGA'] = df_tga['Mean_CDS']
# df_all['No_TAA'] = df_taa['Mean_CDS']

# # df_tag['Mean_CDS_all'] = df_all['Mean_CDS']
# df_all.plot(x="x", y=["Mean_CDS", "No_TAG", "No_TGA", "No_TAA"], figsize=(40, 8))

# # df_tag['Delta'] = df_tag['Mean_CDS'] - df_tag['Mean_CDS_all']

# outfile = './tmp/OFRY01000050-TAG-TGA-TAA.gbk.cds_distr_w1000_s30.txt'
# write_df_to_artemis(df_all, outfile)

=== Working on file ./genbank/Bc11.fasta-TAA.gbk ===
=== Working on file ./genbank/Bc11.fasta-TAG-TAA.gbk ===
=== Working on file ./genbank/Bc11.fasta-TAG-TGA-TAA.gbk ===
=== Working on file ./genbank/Bc11.fasta-TAG-TGA.gbk ===


/home/dec/miniconda3/lib/python3.9/site-packages/Bio/GenBank/Scanner.py:1555: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       contig_1_rc     90579 bp    DNA             PHG\n'
  warnings.warn(
/tmp/ipykernel_9367/2247737863.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_all_stats(infile, outdir), ignore_index=True)
/home/dec/miniconda3/lib/python3.9/site-packages/Bio/GenBank/Scanner.py:1555: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       contig_1_rc     90579 bp    DNA             PHG\n'
  warnings.warn(
/tmp/ipykernel_9367/2247737863.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_all_stats(infile, outdir), ignore_index=True)
/home/dec/miniconda3/lib/python3.9/site-packages/Bio/GenBank/Scann

=== Working on file ./genbank/Bc11.fasta-TAG.gbk ===
=== Working on file ./genbank/Bc11.fasta-TGA-TAA.gbk ===
=== Working on file ./genbank/Bc11.fasta-TGA.gbk ===


/tmp/ipykernel_9367/2247737863.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_all_stats(infile, outdir), ignore_index=True)
/home/dec/miniconda3/lib/python3.9/site-packages/Bio/GenBank/Scanner.py:1555: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       contig_1_rc     90579 bp    DNA             PHG\n'
  warnings.warn(
/tmp/ipykernel_9367/2247737863.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_all_stats(infile, outdir), ignore_index=True)
/home/dec/miniconda3/lib/python3.9/site-packages/Bio/GenBank/Scanner.py:1555: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       contig_1_rc     90579 bp    DNA             PHG\n'
  warnings.warn(
/tmp/ipykernel_9367/2247737863.py:5: FutureWarning: The frame.appe

,infile,id,length,gc,cds_num,cds_avg_len,cds_med_len,cds_tRNA_density
0,./genbank/Bc11.fasta-TAA.gbk,contig_1_rc,90579,0.291458,102,287.245098,114.0,0.960752
1,./genbank/Bc11.fasta-TAG-TAA.gbk,contig_1_rc,90579,0.291458,107,273.140187,112.0,0.958511
2,./genbank/Bc11.fasta-TAG-TGA-TAA.gbk,contig_1_rc,90579,0.291458,101,286.801980,120.0,0.954305
3,./genbank/Bc11.fasta-TAG-TGA.gbk,contig_1_rc,90579,0.291458,99,300.161616,121.0,0.969850
4,./genbank/Bc11.fasta-TAG.gbk,contig_1_rc,90579,0.291458,82,368.036585,169.0,0.974575
5,./genbank/Bc11.fasta-TGA-TAA.gbk,contig_1_rc,90579,0.291458,106,278.254717,118.0,0.965721
6,./genbank/Bc11.fasta-TGA.gbk,contig_1_rc,90579,0.291458,80,376.012500,211.0,0.979995


## Get some stats for a single genome

In [11]:
infiles = glob.glob('./genbank/OFRY01000050*')
outdir = './tmp'
from Bio.SeqRecord import SeqRecord
def get_distribution_of_stops(seqiorec: SeqRecord, window: int = 210, step: int = 1) -> pd.DataFrame:
    """
    Get distribution of STOP codons in a sequence
    :param seqiorec: SeqRecord object
    :param window: window size
    :param step: step size
    :return:
    """

    stops = ['TAA', 'TAG', 'TGA']

    stops_distr = {
        'x': range(1, len(seqiorec.seq) + 1),
        'TAA': [np.NAN]*int(step/2),
        'TAG': [np.NAN]*int(step/2),
        'TGA': [np.NAN]*int(step/2)
    }
    
    i = 0
    while i < len(seqiorec.seq):
        window_seq = seqiorec.seq[i : i + window]
        taa = window_seq.count('TAA')
        tag = window_seq.count('TAG')
        tga = window_seq.count('TGA')
        stops_distr['TAA'].extend([taa]*(step))
        stops_distr['TAG'].extend([tag]*(step))
        stops_distr['TGA'].extend([tga]*(step))
        i += step
        
    i -= step
    left = len(seqiorec.seq) - len(stops_distr['TAA'])
    if left > 0:   
        print('aa')
        stops_distr['TAA'].extend([np.NAN]*left)
        stops_distr['TAG'].extend([np.NAN]*left)
        stops_distr['TGA'].extend([np.NAN]*left)

    for k, v in stops_distr.items():
        print(k, len(v), v[:10], v[-10:])

    return pd.DataFrame(stops_distr)

for infile in infiles[:1]:
    for record in parse_genbank(infile):
        df = get_distribution_of_stops(record,300, 15)
        print(df.tail())

x 96512 range(1, 11) range(96503, 96513)
TAA 96532 [nan, nan, nan, nan, nan, nan, nan, 14, 14, 14] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TAG 96532 [nan, nan, nan, nan, nan, nan, nan, 5, 5, 5] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
TGA 96532 [nan, nan, nan, nan, nan, nan, nan, 2, 2, 2] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/home/dec/miniconda3/lib/python3.9/site-packages/Bio/GenBank/Scanner.py:1555: BiopythonParserWarning: Malformed LOCUS line found - is this correct?
:'LOCUS       OFRY01000050     96512 bp    DNA             PHG\n'
  warnings.warn(


ValueError: All arrays must be of the same length